# How to create a session object, and work with it

In [ ]:
#Auto-reload modules (used to develop functions outside this notebook)
%load_ext autoreload
%autoreload 2

## Imports

In [ ]:
# import modules from 2p-py
import labrotation.file_handling as file_handling
import labrotation.two_photon_session as two_photon_session
import labrotation.belt_processing as belt_processing
import numpy as np
from matplotlib import pyplot as plt

## Set matlab-2p path
matlab is unfortunately still needed, as the belt processing scripts were written in Matlab and have not yet been implemented in Python. See the Test Matlab Integration jupyter notebook on how to set up Matlab in Python.

In [ ]:
matlab_2p_path = file_handling.open_dir("Choose matlab-2p folder!")

## Set file path variables

In [ ]:
nd2_path = file_handling.open_file("Choose Nikon imaging file (.nd2)!")
nd2_meta_path = file_handling.open_file("Choose Nikon metadata file (.txt)!")
labview_path = file_handling.open_file("Choose LabView file (xy.txt, NOT xytime.txt)!")
labview_timestamps_path = labview_path[:-4] + "time.txt"
lfp_path = file_handling.open_file("Choose LFP file (.abf)!")

In [ ]:
if lfp_path == ".":
    lfp_path = None
if labview_path == ".":
    labview_path = None
    labview_timestamps_path = None

## Create session object
Functions such as matching the belt with the nikon recording will be automatically run.

In [ ]:
session = two_photon_session.TwoPhotonSession.init_and_process(nd2_path, nd2_meta_path, labview_path, labview_timestamps_path, lfp_path, matlab_2p_path)

In [ ]:
nik_means = session.mean_fluo

In [ ]:
nik_t = session.nikon_daq_time.values

In [ ]:
nik_sw_t = session.nikon_meta["SW Time [s]"].values

In [ ]:
nik_sw_t = np.array(list(map(lambda x: float(x.replace(",", ".")), nik_sw_t)))

In [ ]:
ar = nik_sw_t[1:] - nik_sw_t[:-1]

In [ ]:
len_rec = nik_sw_t[-1] - nik_sw_t[0]  # the real length of the recording in seconds

In [ ]:
# nik_t: 0 to b, N frames. nik_sw_t: c to d, N frames.
# approximate time as software time (problem: difference between elements is not constant)
nik_t_approx_s = nik_sw_t - nik_sw_t[0]

In [ ]:
nik_t_approx_s

## session is ready to work with!
Check class documentation (or source code) for functionalities

In [ ]:
movement_t, movement_y = session.lfp_movement()  # split components
movement = session.lfp_movement()  # keep components together

In [ ]:

fig = plt.figure(figsize=(18,10))
plt.plot(movement_t, movement_y)  # or: plt.plot(*movement)
plt.show()

In [ ]:
lfp_t, lfp_y = session.lfp_lfp()

In [ ]:
fig = plt.figure(figsize=(18,10))
plt.plot(lfp_t,lfp_y)  # or: plt.plot(*movement)
plt.show()

In [ ]:
fig, axs = plt.subplots(3,1,figsize=(18,18), sharex=True)
axs[0].plot(nik_t_approx_s, nik_means)
axs[0].set_ylim((22,25))
axs[1].plot(lfp_t, lfp_y, color="green")
axs[2].plot(movement_t, movement_y, color="grey")
axs[2].set_ylim((2.0,3.5))
#axs[2].set_xlim((17,20))
plt.show()

In [ ]:
output_folder = file_handling.open_dir("Choose output folder!")

In [ ]:
save_matched_lfp = False
if save_matched_lfp:
    with open(os.path.join(output_folder, "lfp_matched.txt"), "w") as f:
        for i_row in range(len(lfp_t)):
            f.write(f"{lfp_t[i_row]}\t{lfp_y[i_row]}\n")

In [ ]:
save_matched_movement = False
if save_matched_movement:
    with open(os.path.join(output_folder, "movement_matched.txt"), "w") as f:
        for i_row in range(len(movement_t)):
            f.write(f"{movement_t[i_row]}\t{movement_y[i_row]}\n")

In [ ]:
save_nik = False
if save_nik:
    with open(os.path.join(output_folder, "nik_matched.txt"), "w") as f:
        for i_row in range(len(nik_means)):
            f.write(f"{nik_t_approx_s[i_row]}\t{nik_means[i_row]}\n")